In [17]:
# n: personas que no tienen el día que hay que completar
# m: personas que tienen el día que hay que completar

In [5]:
import numpy as np
import polars as pl
from tqdm import tqdm
from cvxpy import Minimize, Variable, Problem, SCS, ECOS, quad_form, diag, sqrt, norm

In [137]:
np.random.seed(18)
n = 6000
mu = np.zeros((n,n))
soc = 15
X = np.random.random((n, 1+ soc))
X[:,0] = np.random.choice([0, 1], size=n, p=[.5, .5])

cov = np.random.rand(soc, soc)
cov = np.dot(cov, cov.T)
invcov = np.linalg.inv(cov)
Q = np.linalg.cholesky(invcov)

for i in tqdm(range(n)):
    XNi = X[i,1:]
    mask = X[:,0] != X[i,0]
    XM  = X[mask,1:]

    num = mask.sum()
    x = Variable(num , name = "x")
    out = (XNi - XM).T
    Q = np.linalg.cholesky(invcov)
    obj = Minimize(norm(((XNi - x @ XM)@Q).T)+ x @ sqrt(diag(quad_form(out, invcov))))
    constr  = [x >= 0, x <= 1, sum(x) == 1]
    val = Problem(obj, constr).solve(solver =  ECOS,verbose = False)
    x_out = x.value.round(5)
    mu[i, mask] = x_out

  9%|▉         | 546/6000 [10:42<1:47:01,  1.18s/it]


KeyboardInterrupt: 

In [37]:
social_vars = ["sexo",
                "edad_anios",
                "quintil_2",
                "quintil_3",
                "quintil_4",
                "quintil_5",
                'nivel_escolaridad_primaria',
                'nivel_escolaridad_secundaria',
                'nivel_escolaridad_técnica',
                'nivel_escolaridad_universitaria',
                "estudia",
                "trabaja",
                'macrozona_norte',
                'macrozona_centro',
                'macrozona_sur',
                "horas_trabajo",
                "n_menores",
                "n_personas"]

data = pl.read_csv("../data/raw/ENUT_PRE_WEEKEND_IMPUTATION.csv",
                    infer_schema_length=100000,
                    null_values = "NA")
data =  (
    data
    .with_columns(quintil=pl.col("quintil").cast(int))
    .to_dummies(columns=["quintil", "macrozona", "nivel_escolaridad"])
    .sort("id_persona")
    .select(["dia_fin_semana"] + social_vars )
    )
data = data[0:500,:]

In [38]:
data = data.to_numpy()
covar = np.cov(data[:,1:].T)

covinv = np.linalg.inv(covar)
Q = np.linalg.cholesky(covinv)

n = len(data)
mu = np.zeros((n,n))

sh_data = data
sh_Q = Q
sh_covinv = covinv
sh_mu = mu

for i in tqdm(range(n)):
    mask = sh_data[:,0] != sh_data[i,0]
    XNi = sh_data[i,1:]
    XM  = sh_data[mask,1:]

    temp = (XNi - XM).T
    x = Variable(mask.sum() , name = "x")

    obj = Minimize(norm(((XNi - x @ XM)@sh_Q).T)+ x @ sqrt(diag(quad_form(temp, sh_covinv))))
    constr  = [x >= 0, x <= 1, sum(x) == 1]

    val = Problem(obj, constr).solve(solver =  ECOS,verbose = False)
    out = x.value.clip(min=0).round(5)
    sh_mu[i, mask] = out

  0%|          | 0/500 [00:00<?, ?it/s]


SolverError: The solver ECOS is not installed.